In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


from helpers.utils import creat_datasets, creating_val_data, reset_graph, grid_serach, read_pickle_file, write_pickle_file
from helpers.utils import creating_train_val_test_datasets, tensorboard_callback,  save_best_model_callback
from helpers.bln_layer import  bln_layer
from helpers.dense_layer import  dense_layer
from helpers.bln_callback import bln_callback 




from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

In [2]:
tf.config.experimental_run_functions_eagerly(True)

In [3]:
tf.__version__

'2.1.0'

### Loading data

In [4]:
random_seed = 100
minibatch = 1
buffersize = 60000
number_valid_sampels = 5000 # number of validation data
epochs = 10
learning_rate = 0.003 

#((25000  - 5000 )/ 1 ) * .2
number_batches_train = 4000 # number of batches to train, each batch of size minibatch parameter
number_batches_valid = 1000 # number of batches to validate, each batch of size minibatch parameter (( 5000 )/ 1 ) * .2
num_classes = 1

max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)


In [5]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen = maxlen)
x_test = sequence.pad_sequences(x_test, maxlen = maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...


C:\Users\Rima\.conda\envs\bln\lib\site-packages\tensorflow_core\python\keras\datasets\imdb.py:129: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\Rima\.conda\envs\bln\lib\site-packages\tensorflow_core\python\keras\datasets\imdb.py:130: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [6]:
x_train, y_train, x_valid, y_valid = creating_val_data(x_train, y_train,
                                                       number_valid_sampels = number_valid_sampels,
                                                       random_seed=random_seed)

train_dataset, valid_dataset, test_dataset = creating_train_val_test_datasets(x_train, y_train,
                                                                              x_test, y_test,
                                                                              x_valid, y_valid, 
                                                                              minibatch = minibatch,
                                                                              buffersize= buffersize,
                                                                              random_seed=random_seed)

In [7]:
train_dataset

<PrefetchDataset shapes: ((1, 80), (1,)), types: (tf.int32, tf.int64)>

# 1. Using Batch Layer Normalization Layer

In [8]:
def BLNLayer_model(inputshape= (80), max_features = 20000, embed_size = 512, random_seed = 100,
                          lstm_unit = 128 , dense_units= 1, batch_size = 10,
                          b_mm = True, b_mv = True,
                          f_mm = False, f_mv = False):
    
   
    # building the model
  
    input_lyr = tf.keras.Input(shape = inputshape, batch_size=batch_size, name = 'input')
    
    x = Embedding(max_features, embed_size,
                  embeddings_initializer=tf.keras.initializers.GlorotUniform(seed=random_seed))(input_lyr)
    
    x = LSTM(lstm_unit, dropout=0.2, recurrent_dropout=0.2,activation='tanh',
            recurrent_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed),
            kernel_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed) ,
            return_sequences=True)(x)
    
    x = bln_layer(stateful = True, batchsize= batch_size, name = 'bn1', 
                  batch_moving_mean = b_mm, batch_moving_var = b_mv,
                  feature_moving_mean = f_mm, feature_moving_var = f_mv)(x) 
    
    x = LSTM(lstm_unit//2, dropout=0.2,  recurrent_dropout=0.2,activation='tanh',
            recurrent_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed),
            kernel_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed) ,
            return_sequences=False)(x)
    
    
    x = bln_layer(stateful = True, batchsize= batch_size, name = 'bn2', 
                  batch_moving_mean = b_mm, batch_moving_var = b_mv,
                  feature_moving_mean = f_mm, feature_moving_var = f_mv)(x) 
    
    x = dense_layer(units = 32, name = 'dense1', random_seed=random_seed)(x)
    x = tf.keras.layers.Activation('tanh')(x)
    x = bln_layer(stateful = True, batchsize= batch_size, name = 'bn3', 
                  batch_moving_mean = b_mm, batch_moving_var = b_mv,
                  feature_moving_mean = f_mm, feature_moving_var = f_mv)(x)
           
    
    output_lyr = dense_layer(units = dense_units, name = 'dense2', random_seed=random_seed)(x)    
    return tf.keras.Model(inputs = [input_lyr], outputs = [output_lyr])

In [9]:
model_bln_layer = BLNLayer_model(inputshape= (80), max_features = max_features, 
                                        embed_size = 512, random_seed = random_seed,
                                        lstm_unit = 128 , dense_units= num_classes,
                                        batch_size = minibatch,
                                        b_mm = False, b_mv = False,
                                        f_mm = False, f_mv = False
                                        )



In [10]:
model_bln_layer.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(1, 80)]                 0         
_________________________________________________________________
embedding (Embedding)        (1, 80, 512)              10240000  
_________________________________________________________________
lstm (LSTM)                  (1, 80, 128)              328192    
_________________________________________________________________
bn1 (bln_layer)              (1, 80, 128)              20898     
_________________________________________________________________
lstm_1 (LSTM)                (1, 64)                   49408     
_________________________________________________________________
bn2 (bln_layer)              (1, 64)                   260       
_________________________________________________________________
dense1 (dense_layer)         (1, 32)                   2080  

# callbacks

In [11]:
# Callbacks for saving best model and tensorboard
folder_name = '/rnn/bln_layer_imdb_batchSize_' + str(minibatch)
 
save_bm_cb = save_best_model_callback(folder_name)
tb_cb = tensorboard_callback(folder_name)

# Callback for resetting moving mean and variances at the end of each epoch
bln_layer_cb = bln_callback()

bln_layer_cb_list = [save_bm_cb, tb_cb, bln_layer_cb]

In [12]:
adam = tf.keras.optimizers.Adam(learning_rate= learning_rate, beta_1=.9, beta_2=.999, epsilon=1e-08)

model_bln_layer.compile(optimizer = adam,
                               loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                               metrics = [tf.keras.metrics.BinaryAccuracy()])

In [13]:
model_bln_layer_history =  model_bln_layer.fit(train_dataset.take(number_batches_train), epochs=epochs,
                                                verbose=1, callbacks=bln_layer_cb_list,
                                                validation_data=valid_dataset.take(number_batches_valid),
                                                shuffle=True)

Train for 4000 steps, validate for 1000 steps
Epoch 1/10
4000/4000 [==============================] - 3617s 904ms/step - loss: 0.6955 - binary_accuracy: 0.5023 - val_loss: 0.6956 - val_binary_accuracy: 0.4880
Epoch 2/10
4000/4000 [==============================] - 3605s 901ms/step - loss: 0.6904 - binary_accuracy: 0.4990 - val_loss: 0.6873 - val_binary_accuracy: 0.5060
Epoch 3/10
4000/4000 [==============================] - 3597s 899ms/step - loss: 0.5846 - binary_accuracy: 0.6610 - val_loss: 0.6869 - val_binary_accuracy: 0.6420
Epoch 4/10
4000/4000 [==============================] - 3599s 900ms/step - loss: 0.4421 - binary_accuracy: 0.7822 - val_loss: 0.6927 - val_binary_accuracy: 0.7020
Epoch 5/10
4000/4000 [==============================] - 3652s 913ms/step - loss: 0.3085 - binary_accuracy: 0.8742 - val_loss: 0.6475 - val_binary_accuracy: 0.7310
Epoch 6/10
4000/4000 [==============================] - 3628s 907ms/step - loss: 0.2044 - binary_accuracy: 0.9215 - val_loss: 0.6940 - val_

In [14]:
del model_bln_layer, save_bm_cb, tb_cb, bln_layer_cb, bln_layer_cb_list
reset_graph()

session is clear


In [15]:
# Select the model with the lowest validation loss
weights_path = './models/rnn/bln_layer_imdb_batchSize_1/best_model.h5'

In [16]:
save_eval_path = "./logs/" + folder_name + '/'+ str(number_batches_train) +"_sorted_evaluation.pkl"
evaluation = grid_serach(BLNLayer_model, test_dataset,
                         batch_size = minibatch, sort=True,
                         save_eval_path = save_eval_path,
                         weights_path = weights_path,
                         loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                         metrics = tf.keras.metrics.BinaryAccuracy())



25000/25000 [==============================] - 7682s 307ms/step - loss: nan - binary_accuracy: 0.5000
{'Bmm_True Bmv_True Fmm_True Fmv_True': [nan, 0.5]}
session is clear
25000/25000 [==============================] - 7722s 309ms/step - loss: nan - binary_accuracy: 0.5000
{'Bmm_True Bmv_True Fmm_True Fmv_True': [nan, 0.5], 'Bmm_True Bmv_True Fmm_True Fmv_False': [nan, 0.5]}
session is clear
25000/25000 [==============================] - 7737s 309ms/step - loss: nan - binary_accuracy: 0.5000
{'Bmm_True Bmv_True Fmm_True Fmv_True': [nan, 0.5], 'Bmm_True Bmv_True Fmm_True Fmv_False': [nan, 0.5], 'Bmm_True Bmv_True Fmm_False Fmv_True': [nan, 0.5]}
session is clear
25000/25000 [==============================] - 7742s 310ms/step - loss: nan - binary_accuracy: 0.5000
{'Bmm_True Bmv_True Fmm_True Fmv_True': [nan, 0.5], 'Bmm_True Bmv_True Fmm_True Fmv_False': [nan, 0.5], 'Bmm_True Bmv_True Fmm_False Fmv_True': [nan, 0.5], 'Bmm_True Bmv_True Fmm_False Fmv_False': [nan, 0.5]}
session is clear
250

25000/25000 [==============================] - 7806s 312ms/step - loss: 0.6570 - binary_accuracy: 0.7311
{'Bmm_True Bmv_True Fmm_True Fmv_True': [nan, 0.5], 'Bmm_True Bmv_True Fmm_True Fmv_False': [nan, 0.5], 'Bmm_True Bmv_True Fmm_False Fmv_True': [nan, 0.5], 'Bmm_True Bmv_True Fmm_False Fmv_False': [nan, 0.5], 'Bmm_True Bmv_False Fmm_True Fmv_True': [0.6940792372179031, 0.5], 'Bmm_True Bmv_False Fmm_True Fmv_False': [0.5833858219459653, 0.7314], 'Bmm_True Bmv_False Fmm_False Fmv_True': [0.6940792372179031, 0.5], 'Bmm_True Bmv_False Fmm_False Fmv_False': [0.6561968348427117, 0.73108], 'Bmm_False Bmv_True Fmm_True Fmv_True': [nan, 0.5], 'Bmm_False Bmv_True Fmm_True Fmv_False': [nan, 0.5], 'Bmm_False Bmv_True Fmm_False Fmv_True': [nan, 0.5], 'Bmm_False Bmv_True Fmm_False Fmv_False': [nan, 0.5], 'Bmm_False Bmv_False Fmm_True Fmv_True': [0.6941002011299133, 0.5], 'Bmm_False Bmv_False Fmm_True Fmv_False': [0.5838858440081776, 0.7314], 'Bmm_False Bmv_False Fmm_False Fmv_True': [0.6941002011

In [17]:
evaluation

[('Bmm_True Bmv_True Fmm_True Fmv_True', [nan, 0.5]),
 ('Bmm_True Bmv_True Fmm_True Fmv_False', [nan, 0.5]),
 ('Bmm_True Bmv_True Fmm_False Fmv_True', [nan, 0.5]),
 ('Bmm_True Bmv_True Fmm_False Fmv_False', [nan, 0.5]),
 ('Bmm_True Bmv_False Fmm_True Fmv_False', [0.5833858219459653, 0.7314]),
 ('Bmm_False Bmv_False Fmm_True Fmv_False', [0.5838858440081776, 0.7314]),
 ('Bmm_True Bmv_False Fmm_False Fmv_False', [0.6561968348427117, 0.73108]),
 ('Bmm_False Bmv_False Fmm_False Fmv_False', [0.6570169957915694, 0.73112]),
 ('Bmm_True Bmv_False Fmm_True Fmv_True', [0.6940792372179031, 0.5]),
 ('Bmm_True Bmv_False Fmm_False Fmv_True', [0.6940792372179031, 0.5]),
 ('Bmm_False Bmv_True Fmm_True Fmv_True', [nan, 0.5]),
 ('Bmm_False Bmv_True Fmm_True Fmv_False', [nan, 0.5]),
 ('Bmm_False Bmv_True Fmm_False Fmv_True', [nan, 0.5]),
 ('Bmm_False Bmv_True Fmm_False Fmv_False', [nan, 0.5]),
 ('Bmm_False Bmv_False Fmm_True Fmv_True', [0.6941002011299133, 0.5]),
 ('Bmm_False Bmv_False Fmm_False Fmv_True'

# 2.Using  Batch Normalization implemented in Keras

In [18]:
def bn_keras_model(inputshape= (80), max_features = 20000, embed_size = 512, random_seed = 100,
                          lstm_unit = 128 , dense_units= 1, batch_size = 60):
    

    
    input_lyr = tf.keras.Input(shape = inputshape, batch_size=batch_size, name = 'input')
    
    x = Embedding(max_features, embed_size,
                  embeddings_initializer=tf.keras.initializers.GlorotUniform(seed=random_seed))(input_lyr)
    
    x = LSTM(lstm_unit, dropout=0.2, recurrent_dropout=0.2, activation='tanh',
            recurrent_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed),
            kernel_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed) ,
            return_sequences=True)(x)
    
    x =  tf.keras.layers.BatchNormalization(momentum = 0.99,  name = 'bn1') (x) 
    
    x = LSTM(lstm_unit//2, dropout=0.2,  recurrent_dropout=0.2, activation='tanh',
            recurrent_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed),
            kernel_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed) ,
            return_sequences=False)(x)
    x =  tf.keras.layers.BatchNormalization(momentum = 0.99,  name = 'bn2') (x) 
    
    x = dense_layer(units = 32, name = 'dense1', random_seed=random_seed)(x)
    x = tf.keras.layers.Activation('tanh')(x)
    x =  tf.keras.layers.BatchNormalization(momentum = 0.99,  name = 'bn3') (x) 

    output_lyr = dense_layer(units = dense_units, name = 'dense2', random_seed=random_seed)(x)    
    return tf.keras.Model(inputs = [input_lyr], outputs = [output_lyr])


In [19]:
model_bn_keras = bn_keras_model(inputshape = (80), max_features = max_features, 
                                embed_size = 512, random_seed = random_seed,
                                lstm_unit = 128 , dense_units= num_classes,
                                batch_size = minibatch)
model_bn_keras.summary()

#### Compiling 
adam = tf.keras.optimizers.Adam(learning_rate= learning_rate, beta_1=.9, beta_2=.999, epsilon=1e-08)
model_bn_keras.compile(optimizer = adam,
                       loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                       metrics = [tf.keras.metrics.BinaryAccuracy()])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(1, 80)]                 0         
_________________________________________________________________
embedding (Embedding)        (1, 80, 512)              10240000  
_________________________________________________________________
lstm (LSTM)                  (1, 80, 128)              328192    
_________________________________________________________________
bn1 (BatchNormalization)     (1, 80, 128)              512       
_________________________________________________________________
lstm_1 (LSTM)                (1, 64)                   49408     
_________________________________________________________________
bn2 (BatchNormalization)     (1, 64)                   256       
_________________________________________________________________
dense1 (dense_layer)         (1, 32)                   2080  

In [20]:
# Callbacks for saving best model and tensorboard
folder_name = '/rnn/bn_Keras_imdb_batchSize_' + str(minibatch)

save_bm_cb = save_best_model_callback(folder_name)
tb_cb = tensorboard_callback(folder_name)


bn_keras_cb_list = [save_bm_cb, tb_cb]

In [21]:
model_bn_keras_history =  model_bn_keras.fit(train_dataset.take(number_batches_train),
                                             epochs = epochs, verbose=1, 
                                             callbacks = bn_keras_cb_list,
                                             validation_data = valid_dataset.take(number_batches_valid),
                                             shuffle = True)

Train for 4000 steps, validate for 1000 steps
Epoch 1/10
4000/4000 [==============================] - 3635s 909ms/step - loss: 0.6938 - binary_accuracy: 0.4985 - val_loss: 1.3533 - val_binary_accuracy: 0.4940
Epoch 2/10
4000/4000 [==============================] - 3659s 915ms/step - loss: 0.6942 - binary_accuracy: 0.4985 - val_loss: 3.0271 - val_binary_accuracy: 0.4980
Epoch 3/10
4000/4000 [==============================] - 3666s 917ms/step - loss: 0.6945 - binary_accuracy: 0.4985 - val_loss: 2.9627 - val_binary_accuracy: 0.4930
Epoch 4/10
4000/4000 [==============================] - 3642s 910ms/step - loss: 0.6944 - binary_accuracy: 0.4985 - val_loss: 2.7060 - val_binary_accuracy: 0.4980
Epoch 5/10
4000/4000 [==============================] - 3672s 918ms/step - loss: 0.6943 - binary_accuracy: 0.4985 - val_loss: 2.4211 - val_binary_accuracy: 0.4840
Epoch 6/10
4000/4000 [==============================] - 3639s 910ms/step - loss: 0.6944 - binary_accuracy: 0.4985 - val_loss: 2.2575 - val_

In [22]:
# Select the model with the lowest validation loss
weights_path = './models/rnn/bn_Keras_imdb_batchSize_1/best_model.h5'
model_bn_keras.load_weights(weights_path)

In [23]:
model_bn_keras_test_history = model_bn_keras.evaluate(test_dataset)

25000/25000 [==============================] - 7799s 312ms/step - loss: 0.9793 - binary_accuracy: 0.4946


In [24]:
del model_bn_keras, save_bm_cb, tb_cb, bn_keras_cb_list 
reset_graph()

session is clear


# 3.Using  Layer normalization  implemented in Keras


In [25]:
def ln_keras_model(inputshape= (80), max_features = 20000, embed_size = 512, random_seed = 100,
                          lstm_unit = 128 , dense_units= 1, batch_size = 60):
    

    
    input_lyr = tf.keras.Input(shape = inputshape, batch_size=batch_size, name = 'input')
    
    x = Embedding(max_features, embed_size,
                  embeddings_initializer=tf.keras.initializers.GlorotUniform(seed=random_seed))(input_lyr)
    
    x = LSTM(lstm_unit, dropout=0.2, recurrent_dropout=0.2, activation='tanh',
            recurrent_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed),
            kernel_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed) ,
            return_sequences=True)(x)
    x =  tf.keras.layers.LayerNormalization()(x) 

    x = LSTM(lstm_unit//2, dropout=0.2,  recurrent_dropout=0.2, activation='tanh',
            recurrent_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed),
            kernel_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed) ,
            return_sequences=False)(x)
    x =  tf.keras.layers.LayerNormalization()(x) 
    
    x = dense_layer(units = 32, name = 'dense1', random_seed=random_seed)(x)
    x = tf.keras.layers.Activation('tanh')(x)
    x =  tf.keras.layers.LayerNormalization()(x) 

    
    output_lyr = dense_layer(units = dense_units, name = 'dense2', random_seed=random_seed)(x)    
    return tf.keras.Model(inputs = [input_lyr], outputs = [output_lyr])



In [26]:
model_ln_keras = ln_keras_model(inputshape= (80), max_features = max_features, 
                                embed_size= 512, random_seed = random_seed,
                                lstm_unit = 128 , dense_units= num_classes,
                                batch_size = minibatch)
model_ln_keras.summary()

#### Compiling 
adam = tf.keras.optimizers.Adam(learning_rate= learning_rate, beta_1=.9, beta_2=.999, epsilon=1e-08)

model_ln_keras.compile(optimizer = adam,
                       loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                       metrics = [tf.keras.metrics.BinaryAccuracy()])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(1, 80)]                 0         
_________________________________________________________________
embedding (Embedding)        (1, 80, 512)              10240000  
_________________________________________________________________
lstm (LSTM)                  (1, 80, 128)              328192    
_________________________________________________________________
layer_normalization (LayerNo (1, 80, 128)              256       
_________________________________________________________________
lstm_1 (LSTM)                (1, 64)                   49408     
_________________________________________________________________
layer_normalization_1 (Layer (1, 64)                   128       
_________________________________________________________________
dense1 (dense_layer)         (1, 32)                   2080  

In [27]:
# Callbacks for saving best model and tensorboard
folder_name = '/rnn/ln_Keras_imdb_batchSize_' + str(minibatch) 

save_bm_cb = save_best_model_callback(folder_name)
tb_cb = tensorboard_callback(folder_name)

ln_keras_cb_list = [save_bm_cb, tb_cb]

In [28]:
model_ln_keras_history =  model_ln_keras.fit(train_dataset.take(number_batches_train),
                                             epochs=epochs, verbose=1, 
                                             callbacks=ln_keras_cb_list,
                                             validation_data=valid_dataset.take(number_batches_valid),
                                             shuffle=True)

Train for 4000 steps, validate for 1000 steps
Epoch 1/10
4000/4000 [==============================] - 3571s 893ms/step - loss: 0.7074 - binary_accuracy: 0.5040 - val_loss: 0.6910 - val_binary_accuracy: 0.4880
Epoch 2/10
4000/4000 [==============================] - 3584s 896ms/step - loss: 0.6490 - binary_accuracy: 0.5690 - val_loss: 0.6834 - val_binary_accuracy: 0.4880
Epoch 3/10
4000/4000 [==============================] - 3578s 895ms/step - loss: 0.5711 - binary_accuracy: 0.6933 - val_loss: 0.6880 - val_binary_accuracy: 0.6240
Epoch 4/10
4000/4000 [==============================] - 3559s 890ms/step - loss: 0.4699 - binary_accuracy: 0.7918 - val_loss: 0.6675 - val_binary_accuracy: 0.6920
Epoch 5/10
4000/4000 [==============================] - 3578s 894ms/step - loss: 0.3705 - binary_accuracy: 0.8510 - val_loss: 0.6881 - val_binary_accuracy: 0.6950
Epoch 6/10
4000/4000 [==============================] - 3445s 861ms/step - loss: 0.3064 - binary_accuracy: 0.8752 - val_loss: 0.7601 - val_

In [34]:
# Select the model with the lowest validation loss

weights_path = './models/rnn/ln_Keras_imdb_batchSize_1/best_model.h5'

model_ln_keras.load_weights(weights_path)

In [35]:
model_ln_keras_test_history = model_ln_keras.evaluate(test_dataset)

 1427/25000 [>.............................] - ETA: 2:03:49 - loss: 0.6841 - binary_accuracy: 0.6896ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "C:\Users\Rima\.conda\envs\bln\lib\site-packages\tensorflow_core\python\keras\backend.py", line 4167, in <genexpr>
    ta.write(time, out) for ta, out in zip(output_ta_t, flat_output))  File "C:\Users\Rima\.conda\envs\bln\lib\site-packages\tensorflow_core\python\util\tf_should_use.py", line 237, in wrapped
    error_in_function=error_in_function)
25000/25000 [==============================] - 8517s 341ms/step - loss: 0.6875 - binary_accuracy: 0.6842


In [36]:
del model_ln_keras,save_bm_cb, ln_keras_cb_list
reset_graph()

session is clear


# 4.writing pickle files

In [37]:
write_pickle_file('./logs/rnn/ln_Keras_imdb_batchSize_1/model_ln_keras_history_1.pickle', model_ln_keras_history.history)
write_pickle_file('./logs/rnn/ln_Keras_imdb_batchSize_1/model_ln_keras_test_history_1.pickle', model_ln_keras_test_history)

write_pickle_file('./logs/rnn/bn_Keras_imdb_batchSize_1/model_bn_keras_history_1.pickle', model_bn_keras_history.history)
write_pickle_file('./logs/rnn/bn_Keras_imdb_batchSize_1/model_bn_keras_test_history_1.pickle', model_bn_keras_test_history)

write_pickle_file('./logs/rnn/bln_layer_imdb_batchSize_1/model_bln_keras_history_1.pickle', model_bln_layer_history.history)
write_pickle_file('./logs/rnn/bln_layer_imdb_batchSize_1/model_bln_keras_test_history_1.pickle', evaluation)